In [22]:
import pandas as pd
import requests
import pandas as pd
import json
import os
import numpy as np
from scipy.stats import poisson


API_KEY = os.getenv("API_KEY")
url = 'https://data-service.beatthebookie.blog/data'
headers = {"x-api-key": API_KEY}
params = {'division':'Premier League'}
response = requests.get(url, headers=headers, params=params)
json_str = response.content.decode('utf-8')
df = pd.read_json(json_str)

df['match_date'] = pd.to_datetime(df['match_date'])
df = df[df["match_date"] > '2023-01-01']

print(df[["match_date", "home_team", "away_team", "home_goals", "home_xgoals", "away_goals", "away_xgoals"]].tail())

     match_date    home_team  away_team  home_goals  home_xgoals  away_goals  \
3815 2024-08-24       Fulham  Leicester           2      1.96218           1   
3816 2024-08-24     Man City    Ipswich           4      3.07818           1   
3817 2024-08-25  Bournemouth  Newcastle           1      2.53513           1   
3818 2024-08-25       Wolves    Chelsea           2      2.41810           6   
3819 2024-08-25    Liverpool  Brentford           2      2.71877           0   

      away_xgoals  
3815     0.857005  
3816     0.479999  
3817     1.892870  
3818     2.029350  
3819     0.457239  


C:\Users\Owner\AppData\Local\Temp\ipykernel_7220\3264941509.py:16: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_str)


In [23]:
def decay(xi, t):
    return np.exp(-xi * t)

In [24]:
def rho_correction(goals_home, goals_away, home_exp, away_exp, rho):
    if goals_home == 0 and goals_away == 0:
        return 1 - (home_exp * away_exp * rho)
    elif goals_home == 0 and goals_away == 1:
        return 1 + (home_exp * rho)
    elif goals_home == 1 and goals_away == 0:
        return 1 + (away_exp * rho)
    elif goals_home == 1 and goals_away == 1:
        return 1 - rho
    else:
        return 1.0

In [25]:
def log_likelihood(
    goals_home_observed,
    goals_away_observed,
    home_attack,
    home_defence,
    away_attack,
    away_defence,
    home_advantage,
    rho,
    weight
):
    goal_expectation_home = np.exp(home_attack + away_defence + home_advantage)
    goal_expectation_away = np.exp(away_attack + home_defence)

    home_llk = poisson.pmf(goals_home_observed, goal_expectation_home)
    away_llk = poisson.pmf(goals_away_observed, goal_expectation_away)
    adj_llk = rho_correction(
        goals_home_observed,
        goals_away_observed,
        goal_expectation_home,
        goal_expectation_away,
        rho,
    )

    if goal_expectation_home < 0 or goal_expectation_away < 0 or adj_llk < 0:
        return 10000

    log_llk = weight * (np.log(home_llk) + np.log(away_llk) + np.log(adj_llk))

    return -log_llk

In [26]:
from pprint import pprint
import numpy as np
from scipy.optimize import minimize
from scipy.stats import poisson


def fit_poisson_model(df, xi=0.0001):
    teams = np.sort(np.unique(np.concatenate([df["home_team"], df["away_team"]])))
    n_teams = len(teams)
    
    df["days_since"] = (df["match_date"].max() - df["match_date"]).dt.days
    df["weight"] = decay(xi, df["days_since"])

    params = np.concatenate(
        (
            np.random.uniform(0.5, 1.5, (n_teams)),  # attack strength
            np.random.uniform(0, -1, (n_teams)),  # defence strength
            [0.25],  # home advantage
            [-0.1], # rho
        )
    )

    def _fit(params, df, teams):
        attack_params = dict(zip(teams, params[:n_teams]))
        defence_params = dict(zip(teams, params[n_teams : (2 * n_teams)]))
        home_advantage = params[-2]
        rho = params[-1]

        llk = list()
        for idx, row in df.iterrows():
            tmp = log_likelihood(
                row["home_goals"],
                row["away_goals"],
                attack_params[row["home_team"]],
                defence_params[row["home_team"]],
                attack_params[row["away_team"]],
                defence_params[row["away_team"]],
                home_advantage,
                rho,
                row["weight"]
            )
            llk.append(tmp)

        return np.sum(llk)

    options = {
        "maxiter": 100,
        "disp": False,
    }

    constraints = [{"type": "eq", "fun": lambda x: sum(x[:n_teams]) - n_teams}]

    res = minimize(
        _fit,
        params,
        args=(df, teams),
        constraints=constraints,
        options=options,
    )

    model_params = dict(
        zip(
            ["attack_" + team for team in teams]
            + ["defence_" + team for team in teams]
            + ["home_adv", "rho"],
            res["x"],
        )
    )

    print("Log Likelihood: ", res["fun"])

    return model_params

In [27]:
model_params = fit_poisson_model(df, xi=0.001)
pprint(model_params)

C:\Users\Owner\AppData\Local\Temp\ipykernel_7220\3438154607.py:28: RuntimeWarning: divide by zero encountered in log
  log_llk = weight * (np.log(home_llk) + np.log(away_llk) + np.log(adj_llk))
C:\Users\Owner\AppData\Local\Temp\ipykernel_7220\3438154607.py:28: RuntimeWarning: divide by zero encountered in log
  log_llk = weight * (np.log(home_llk) + np.log(away_llk) + np.log(adj_llk))
C:\Users\Owner\AppData\Local\Temp\ipykernel_7220\3438154607.py:28: RuntimeWarning: divide by zero encountered in log
  log_llk = weight * (np.log(home_llk) + np.log(away_llk) + np.log(adj_llk))
C:\Users\Owner\AppData\Local\Temp\ipykernel_7220\3438154607.py:28: RuntimeWarning: divide by zero encountered in log
  log_llk = weight * (np.log(home_llk) + np.log(away_llk) + np.log(adj_llk))
C:\Users\Owner\AppData\Local\Temp\ipykernel_7220\3438154607.py:28: RuntimeWarning: divide by zero encountered in log
  log_llk = weight * (np.log(home_llk) + np.log(away_llk) + np.log(adj_llk))
C:\Users\Owner\AppData\Local\T

Log Likelihood:  1341.2816917255536
{'attack_Arsenal': 1.4795549120357818,
 'attack_Aston Villa': 1.2438941643962187,
 'attack_Bournemouth': 0.8928214666417339,
 'attack_Brentford': 1.029335861906427,
 'attack_Brighton': 1.1337716074035127,
 'attack_Burnley': 0.7429387158274882,
 'attack_Chelsea': 1.2268650289864804,
 'attack_Crystal Palace': 0.9563828495607469,
 'attack_Everton': 0.6446939341262722,
 'attack_Fulham': 0.9712657878401693,
 'attack_Ipswich': 0.4067124298935615,
 'attack_Leeds': 0.8193415848553084,
 'attack_Leicester': 0.8310838778211198,
 'attack_Liverpool': 1.402732173378351,
 'attack_Luton': 0.9887170160568373,
 'attack_Man City': 1.540283413559404,
 'attack_Man United': 1.048010040772678,
 'attack_Newcastle': 1.3617771519002762,
 "attack_Nott'm Forest": 0.8851100682582066,
 'attack_Sheffield United': 0.6049460167263805,
 'attack_Southampton': 0.554481748571359,
 'attack_Tottenham': 1.30700445142699,
 'attack_West Ham': 1.075714056383626,
 'attack_Wolves': 0.8525616416

In [32]:
def predict(params, home_team, away_team):
    home_attack = params["attack_" + home_team]
    home_defence = params["defence_" + home_team]
    away_attack = params["attack_" + away_team]
    away_defence = params["defence_" + away_team]
    home_advantage = params["home_adv"]
    rho = params["rho"]

    home_goal_expectation = np.exp(home_attack + away_defence + home_advantage)
    away_goal_expectation = np.exp(away_attack + home_defence)

    home_probs = poisson.pmf(range(10), home_goal_expectation)
    away_probs = poisson.pmf(range(10), away_goal_expectation)

    m = np.outer(home_probs, away_probs)

    m[0, 0] *= 1 - home_goal_expectation * away_goal_expectation * rho
    m[0, 1] *= 1 + home_goal_expectation * rho
    m[1, 0] *= 1 + away_goal_expectation * rho
    m[1, 1] *= 1 - rho    

    home = np.sum(np.tril(m, -1))
    draw = np.sum(np.diag(m))
    away = np.sum(np.triu(m, 1))

    # Calculate the probability of a clean sheet for the home team (away team scores 0)
    home_clean_sheet_prob = m[:, 0].sum() * 100

    # Calculate the probability of a clean sheet for the away team (home team scores 0)
    away_clean_sheet_prob = m[0, :].sum() * 100

    return home, draw, away, home_clean_sheet_prob, away_clean_sheet_prob, home_goal_expectation, away_goal_expectation

In [40]:
predict(model_params, "Tottenham", "Arsenal")

(0.20848242086124008,
 0.19697533823419539,
 0.5944487997073633,
 11.335540279889928,
 29.578371231157853,
 1.2180340059089105,
 2.17722658034776)